In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import plot_cost_history
from IPython.display import display

## Load previous results

### load price elasticity

In [2]:
categories_enc =  {'Aquatic Roots and Tubers':'1', 'Cauliflower':'2' ,'Chili Peppers':'3', 'Edible Mushrooms':'4', 'Leafy Greens':'5', 'Solanaceous Vegetables':'6'}

price_elasticity = []

for category in categories_enc.keys():
    category = category.replace(" ", "_")
    coef_df = pd.read_csv(f'../results/reg_coef_{category}.csv',index_col=0)

    paddings_ = list(set(range(1,7)) - set([int(idx.split('_')[-1]) for idx in coef_df.index if 'avg_sale_price' in idx]))
    for padding in paddings_:
        coef_df.loc[f'avg_sale_price_{padding}','coef'] = 0
    coef_df = coef_df.loc[[idx for idx in coef_df.index if 'avg_sale_price' in idx],['coef']]
    coef_df.sort_index(inplace=True)
    price_elasticity.append(coef_df['coef'].values)

price_elasticity = np.array(price_elasticity)
print(price_elasticity.shape)
# price_elasticity[i][j] =  the demand elasticity of category i to price of category j

(6, 6)


### load prediction of sale volume, wholesale price

In [3]:
pred_sale_volume = np.load('../results/pred_sale_volume.npy')
pred_wholesale_price = np.load('../results/pred_wholesale_price.npy')
print(pred_sale_volume.shape, pred_wholesale_price.shape)
# pred_XX[i][j] = the predicted sale volume/wholesale price of day i of category j

(7, 6) (7, 6)


In [4]:
daily_category_sales = pd.read_csv('../data/daily_category_sales.csv',index_col=0, header=[0,1])
recent_sales = daily_category_sales.iloc[-7:,:]
recent_prices = (recent_sales['cost_sum'] + recent_sales['sales_profit_sum']) / recent_sales['quantity_sum']
avg_recent_prices = recent_prices.mean().values

discounts = [0.7]*6
spoilage_rates = [0.1365, 0.1551, 0.0924, 0.0945, 0.1283, 0.0668]



2023-11-13 13:31:31,163 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-11-13 13:31:31,165 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


### display profit and profit rate of last week

In [118]:
display(recent_sales['sales_profit_sum'].sum(axis=1))
display(recent_sales['avg_profit_rate'])

date
2023-06-24    800.84431
2023-06-25    579.26514
2023-06-26    568.17112
2023-06-27    614.58183
2023-06-28    623.81306
2023-06-29    741.08815
2023-06-30    802.88091
dtype: float64

category_name,Aquatic Roots and Tubers,Cauliflower,Chili Peppers,Edible Mushrooms,Leafy Greens,Solanaceous Vegetables
date,,,,,,
2023-06-24,0.405043,0.610105,0.731536,0.457419,0.472986,0.602299
2023-06-25,0.359395,0.717791,0.776939,0.486291,0.394725,0.592299
2023-06-26,0.382010,0.763224,0.846159,0.495508,0.543217,0.574084
2023-06-27,0.346077,0.538462,0.828405,0.419364,0.467015,0.695391
2023-06-28,0.359849,0.568282,0.801574,0.489069,0.378821,0.614452
2023-06-29,0.395814,0.483607,0.719349,0.468467,0.433461,0.666091
2023-06-30,0.347812,0.455646,0.645836,0.450072,0.636629,0.725837


## Set up rules

### Adjust demand volume with estimated Price elasticity of demand

$$
\Delta \ln Q_{k} = \sum\limits_{i=1}^m \beta_{ki} \Delta \ln P_{i} \ ,\hspace{10pt} \text{i.e.} \ \ln \frac{Q_k}{Q_{base,k}} = \sum\limits_{i=1}^m \beta_{ki} \ln \frac{P_{i}}{P_{base,i}} \hspace{5pt} (k=1,2,\cdots,m)
$$
where $Q_{base}$ is the predict value of LSTM, and $P_{base}$ is last week's average selling price of the category

In [5]:
def adjust_demand(q_base, p_curr, p_base=avg_recent_prices, p_elasticity=price_elasticity):
    """
    p_base: the base price
    p_curr: the current price
    q_base: the base demand
    p_elasticity: the price elasticity
    """
    q_log_delta = np.matmul(p_elasticity, np.log(p_curr/p_base))

    return q_base * np.exp(q_log_delta)

### Calculate daily profit

In [78]:
def daily_profit(profit_rate, q_buyin, 
                pred_sale_volume=pred_sale_volume, 
                pred_wholesale_price=pred_wholesale_price, 
                discounts=discounts, 
                spoilage_rates=spoilage_rates):
    """
    x: the price of each category
    pred_sale_volume: the predicted sale volume of each category
    pred_wholesale_price: the predicted wholesale price of each category
    discounts: the discount rate of each category
    spoilage_rates: the spoilage rate of each category
    """


    p_buyin = pred_wholesale_price[day] #外部变量day控制预测的是第几天
    q_demand_pred = pred_sale_volume[day]

    p_sale = np.multiply(p_buyin, np.ones(6)+profit_rate)
    p_sale_discount = np.multiply(p_sale, discounts)
    q_demand_adjusted = adjust_demand(q_base=q_demand_pred, p_curr=p_sale)

    q_buyin_normal = np.multiply(q_buyin, (np.array([1]*6)-spoilage_rates))
    q_buyin_discount = np.multiply(q_buyin, spoilage_rates)

    profit = 0
    for idx in range(6):
        cost = p_buyin[idx]*q_buyin[idx]
        if q_buyin[idx] <= q_demand_adjusted[idx]:
            profit += (p_sale[idx]*q_buyin_normal[idx] + p_sale_discount[idx]*q_buyin_discount[idx]) - cost
        else:
            profit += p_sale[idx]*np.multiply(q_demand_adjusted, (np.array([1]*6)-spoilage_rates))[idx] + \
                p_sale_discount[idx]*np.multiply(q_demand_adjusted, spoilage_rates)[idx] - cost
        

    return profit

In [128]:
profit_rate = test_profit_rate
q_buyin = test_q_buyin

p_buyin = pred_wholesale_price[day] #外部变量day控制预测的是第几天
q_demand_pred = pred_sale_volume[day]

p_sale = np.multiply(p_buyin, np.ones(6)+profit_rate)
p_sale_discount = np.multiply(p_sale, discounts)
q_demand_adjusted = adjust_demand(q_base=q_demand_pred, p_curr=p_sale)

q_buyin_normal = np.multiply(q_buyin, (np.array([1]*6)-spoilage_rates))
q_buyin_discount = np.multiply(q_buyin, spoilage_rates)

profit = 0
for idx in range(6):
    cost = p_buyin[idx]*q_buyin[idx]
    if q_buyin[idx] <= q_demand_adjusted[idx]:
        profit += (p_sale[idx]*q_buyin_normal[idx] + p_sale_discount[idx]*q_buyin_discount[idx]) - cost
    else:
        profit += p_sale[idx]*np.multiply(q_demand_adjusted, (np.array([1]*6)-spoilage_rates))[idx] + \
            p_sale_discount[idx]*np.multiply(q_demand_adjusted, spoilage_rates)[idx] - cost

print(f'q_demand_pred: {q_demand_pred}\n')
print(f'q_demand_adjusted: {q_demand_adjusted}\n')
print(f'q_buyin: {q_buyin}\n')
print(f'q_buyin_normal: {q_buyin_normal}\n')
print(f'q_buyin_discount: {q_buyin_discount}\n')
print(f'p_buyin: {p_buyin}\n')
print(f'p_sale: {p_sale}\n')
print(f'profit: {profit}\n')

q_demand_pred: [  3.6777308  17.407774   73.02979    33.26445   119.091      19.577618 ]

q_demand_adjusted: [  3.27243156  16.89255286  69.20590397  29.11879904 119.27879569
  20.66781548]

q_buyin: [  3.6409535  17.233696   72.29949    32.931805  117.90009    19.381842 ]

q_buyin_normal: [  3.14396338  14.56074974  65.61901883  29.81974912 102.77351114
  18.08713464]

q_buyin_discount: [ 0.49699016  2.67294625  6.68047305  3.11205554 15.12658194  1.29470702]

p_buyin: [12.260101   8.125391   3.277806   3.6586807  3.0906625  4.21375  ]

p_sale: [17.16414185 13.00062561  5.90005088  6.21975715  5.25412621  7.16337481]

profit: 579.7172604747877



In [111]:
def objective_func(X): # X: (n_particles, 12) = (particles, concat(profit_rate, buyin_quantity))
    profit_list = []
    
    for i in range(n_particles):
        particle_pos = X[i]

        profit_rate = particle_pos[:6]
        q_buyin = particle_pos[6:]
        profit = daily_profit(profit_rate, q_buyin)
        profit_list.append(-profit)
    
    return profit_list

In [ ]:
ini_pos [] # initialize wholesale prices and sale volume
ini_pose

n_particles = 1000
n_dimensions = 12
lower_bound = np.array([])
upper_bound = np.array([])
bounds = (lower_bound, upper_bound)



# 一部分粒子的初始位置给定，另一部分初始位置随机
weight_ini = 0.3 # 给定的初始值占总粒子的比例
pos_given = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*weight_ini), n_dimensions))
pos_given = 0.8 * pos_given + 0.2 * ini_pos
pos_random = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*(1-weight_ini)), n_dimensions))

initial_pos = np.vstack((pos_given, pos_random))

In [ ]:
options = {'c1': 0.5, 'c2': 0.5, 'w': 0.6} # 个人 社会 继承因子， PSO算法的参数
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, 
                                    dimensions=n_dimensions, 
                                    options=options, 
                                    bounds=bounds,
                                    init_pos=initial_pos)

best_position, best_cost = optimizer.optimize(objective_func, iters=1000, verbose=True)

fig, ax = plt.subplots()
beautiful(ax)
plot_cost_history(cost_history=optimizer.cost_history, title="目标函数(-利润)变化曲线", ax=ax)
plt.show()

In [ ]:
def pre_sale_amount(sale_price, sale_lstm):
    list_sale_amount = []
    for idx in range(6):
        price = sale_price[idx]
        sale_volume = sale_lstm[idx]
        sale_amount = modify(sale_volume, price, idx)
    
    return list_sale_amount

